In [2]:
import neuroglancer
import numpy as np
import os
import sys
import navis

# add path to ac_ngl to sys.path
sys.path.append('/home/wanqing.yu/AC_Project/ac_visualization/')
from ac_ngl import *

In [3]:
# source_path = '/ACdata/Users/wanqing/exaSPIM/forOlga/Result of tile_x_0002_y_0001_z_0000_ch_488_mip2-8bit-zcut.tif'
source_path = '/ACdata/Users/wanqing/exaSPIM/exaSPIM_H17.24.006_2023-09-22_14-52-45/exaSPIM/mip0/test.tif'

# out_path = '/ACdata/Users/wanqing/exaSPIM/precomputed'
out_path = '/ACdata/Users/wanqing/exaSPIM/test/precomputed/'

Generate ExASPIM precomputed skeletons

In [4]:
# s_range = [53,55]#[52,54]
indir = '/ACdata/Users/wanqing/exaSPIM/exaSPIM_H17.24.006_2023-09-22_14-52-45/exaSPIM/mip2/x2y1z0/skeleton/skeletons_10' #'/home/wanqing.yu/EM Connectome/axon_analysis/skeletons/skel_axon/' #/home/wanqing.yu/EM Connectome/axon_analysis/skeletons/rid_sid_cleaned/' #rid_only/' # '/ACdata/Users/connorl/Skeletons/For_Kevin/S32_Pos52,53,54_MIP0/' ###skeletons
outdir = '/ACdata/Users/wanqing/Neuroglancer/ExASPIM/' #skel_axon/'
precomputed_image = '/ACdata/Users/wanqing/exaSPIM/precomputed/'

mip = 0
skel_trans = [0,0,0] #[23000,0,0]
scale_im = [2962,2961,4000] # nm
dim_im = neuroglancer.CoordinateSpace(
                names=['z', 'y', 'x', 't'],
                units='nm',
                scales=scale_im,
            )

dim_seg = neuroglancer.CoordinateSpace(
                names=['z', 'y', 'x', 't'],
                units='nm',
                scales=scale_im,
            )

In [19]:
# ###Create precomputed volume for skeletons
# pos_dir = indir
# os.makedirs(outdir, exist_ok=True)
# generate_ngl_segmentation_empty([288,288,21586], pos_dir, outdir, [512, 512, 64], [406, 406, 1997.72])
# generate_ngl_skeletons(pos_dir, outdir,match_fname=False)

Generate neuroglancer link

In [5]:
###Load zarr and skeletons into neuroglancer instance
ip = 'localhost' 
port = 9998
neuroglancer.set_server_bind_address(bind_address=ip,bind_port=port)
viewer=neuroglancer.Viewer()

rotation_matrix_skeletons = np.array([[0, 0, 1, 0],
                                      [0, 1, 0, 0],
                                      [1, 0, 0, 0]])

#translation_vector_skeletons = [0, 0, 340]  # Adjust these values as needed

# Combine rotation and translation
#transformation_matrix_skeletons = rotation_matrix_skeletons + np.array([translation_vector_skeletons])


#load image and skeletons
###load image and skeletons
tr_im = neuroglancer.CoordinateSpaceTransform(input_dimensions = dim_im, output_dimensions =  dim_im) 
tr_seg = neuroglancer.CoordinateSpaceTransform(input_dimensions = dim_seg, output_dimensions =  dim_seg,
                                                  matrix = rotation_matrix_skeletons)

with viewer.txn() as s:
    s.layers['ExASPIM'] = neuroglancer.ImageLayer(source=['precomputed://http://bigkahuna.corp.alleninstitute.org/' + precomputed_image])
    # s.layers['ExASPIM'].layer.source[0].transform  = tr_im

with viewer.txn() as s:
    s.layers['skeletons'] = neuroglancer.SegmentationLayer(source=['precomputed://http://bigkahuna.corp.alleninstitute.org/' + outdir +'skeletons/'])
    s.layers['skeletons'].layer.source[0].transform  = tr_seg
    
print(viewer)

http://localhost:9998/v/d9b9e86858fec61ed9a639b5fdbba94bc591bd0b/


In [17]:
# ###Create shareable neuroglancer link
# view = s.to_json()
# make_neuroglancer_url_vneurodata(view)

http://bigkahuna.corp.alleninstitute.org/neuroglancer/#!https://json.neurodata.io/v1?NGStateID=-3onGZrN-YlSDA


In [6]:
## loading all skeletons 
skel_all_navis = navis.read_swc('/ACdata/Users/wanqing/exaSPIM/exaSPIM_H17.24.006_2023-09-22_14-52-45/exaSPIM/mip2/x2y1z0/skeleton/skeletons_10/')
skel_all_navis

Importing:   0%|          | 0/6004 [00:00<?, ?it/s]

,type,name,n_nodes,n_connectors,n_branches,n_leafs,cable_length,soma,units,created_at,origin
0,navis.TreeNeuron,3358,18,None,0,1,144.625000,None,1 dimensionless,2024-08-16 11:26:23.074996,/ACdata/Users/wanqing/exaSPIM/exaSPIM_H17.24.0...
1,navis.TreeNeuron,2518,22,None,0,1,174.097763,None,1 dimensionless,2024-08-16 11:26:23.075117,/ACdata/Users/wanqing/exaSPIM/exaSPIM_H17.24.0...
...,...,...,...,...,...,...,...,...,...,...,...
6002,navis.TreeNeuron,3547,17,None,0,1,141.712128,None,1 dimensionless,2024-08-16 11:26:26.529009,/ACdata/Users/wanqing/exaSPIM/exaSPIM_H17.24.0...
6003,navis.TreeNeuron,3935,15,None,0,1,134.598907,None,1 dimensionless,2024-08-16 11:26:26.522938,/ACdata/Users/wanqing/exaSPIM/exaSPIM_H17.24.0...


In [7]:
id = 1
#skel_navis = skel_all_navis[skel_all_navis.name == str(id+1)].copy() 
skel_navis = navis.read_swc('/ACdata/Users/wanqing/exaSPIM/exaSPIM_H17.24.006_2023-09-22_14-52-45/exaSPIM/mip2/x2y1z0/skeleton/skeletons_10/0002.swc')
skel_navis.nodes


,node_id,label,x,y,z,radius,parent_id,type
0,1,2,1743.849976,1924.849976,246.000000,3.2480,-1,root
1,2,2,1736.180054,1924.000000,249.000000,6.4960,1,slab
2,3,2,1727.880005,1923.000000,252.117996,6.4960,2,slab
3,4,2,1720.449951,1921.449951,255.000000,6.4960,3,slab
4,5,2,1713.199951,1920.800049,257.802002,6.4960,4,slab
...,...,...,...,...,...,...,...,...
144,145,2,900.346985,2617.649902,337.000000,7.2627,144,slab
145,146,2,898.057983,2624.939941,334.058014,6.4960,145,slab
146,147,2,894.570984,2632.429932,331.000000,6.4960,146,slab
147,148,2,891.000000,2639.800049,328.000000,6.4960,147,slab


In [8]:
# switch x and z in skel_navis.nodes
tmp_x = skel_navis.nodes.x
skel_navis.nodes.x = skel_navis.nodes.z
skel_navis.nodes.z = tmp_x
skel_navis.nodes

,node_id,label,x,y,z,radius,parent_id,type
0,1,2,246.000000,1924.849976,1743.849976,3.2480,-1,root
1,2,2,249.000000,1924.000000,1736.180054,6.4960,1,slab
2,3,2,252.117996,1923.000000,1727.880005,6.4960,2,slab
3,4,2,255.000000,1921.449951,1720.449951,6.4960,3,slab
4,5,2,257.802002,1920.800049,1713.199951,6.4960,4,slab
...,...,...,...,...,...,...,...,...
144,145,2,337.000000,2617.649902,900.346985,7.2627,144,slab
145,146,2,334.058014,2624.939941,898.057983,6.4960,145,slab
146,147,2,331.000000,2632.429932,894.570984,6.4960,146,slab
147,148,2,328.000000,2639.800049,891.000000,6.4960,147,slab


In [9]:
# rotate axis
root_pos = [skel_navis.nodes.x[0], skel_navis.nodes.y[0], skel_navis.nodes.z[0]]
root_pos

[246.0, 1924.85, 1743.85]

In [10]:
# make a cetain segment visible
with viewer.txn() as s:
  s.layers['segmentation'].segments = [id]
  s.voxel_coordinates = root_pos

In [11]:
import copy
import time

def interpolate_to(final_state, frames_per_second=5, seconds=1):
    total_frames = int(round(seconds * frames_per_second))
    initial_state = viewer.state
    for frame_i in range(total_frames):
        t = frame_i / total_frames
        viewer.set_state(
            neuroglancer.ViewerState.interpolate(initial_state, final_state, t)
        )
        time.sleep(1 / frames_per_second)
    viewer.set_state(final_state)

def move_by(offset, **kwargs):
    final_state = copy.deepcopy(viewer.state)
    final_state.voxel_coordinates += offset
    interpolate_to(final_state, **kwargs)

def move_to(final_voxel_coordinates, final_orientation, **kwargs):
    final_state = copy.deepcopy(viewer.state)
    final_state.voxel_coordinates = final_voxel_coordinates
    final_state.crossSectionOrientation = final_orientation
    final_state.projectionOrientation = final_orientation
    interpolate_to(final_state, **kwargs)

def do_move_by():
    move_by([100, 100, 100], seconds=1, frames_per_second=10)

def do_move_to(final_voxel_coordinates, final_orientation, seconds=1, frames_per_second=10):
    move_to(final_voxel_coordinates, final_orientation = final_orientation, seconds=seconds, frames_per_second=frames_per_second)

def normalize(v):
    norm = np.linalg.norm(v)
    return v / norm if norm > 0 else v

def surface_normal_to_quaternion(normal):
    normal = normalize(normal)
    up_vector = np.array([0, 0, 1]) # need to update up_vector based on the previous step
    axis = np.cross(up_vector, normal)
    axis = normalize(axis)
    angle = np.arccos(np.dot(up_vector, normal))
    
    qx = axis[0] * np.sin(angle / 2)
    qy = axis[1] * np.sin(angle / 2)
    qz = axis[2] * np.sin(angle / 2)
    qw = np.cos(angle / 2)
    
    return [qx, qy, qz, qw]


import math

def angle_between_vectors(v1, v2):
    dot_product = np.dot(v1, v2)
    magnitude_v1 = np.linalg.norm(v1)
    magnitude_v2 = np.linalg.norm(v2)
    cos_angle = dot_product / (magnitude_v1 * magnitude_v2)
    cos_angle = np.clip(cos_angle, -1, 1)  # Ensure within valid range for arccos
    angle_radians = np.arccos(cos_angle)
    angle_degrees = np.degrees(angle_radians)
    return angle_degrees

def flip_vectors_if_necessary(skel_dp_vect):
    """Flip the direction of vectors if the angle with the previous vector is > 150 degrees."""
    for i in range(1, len(skel_dp_vect)):
        angle = angle_between_vectors(skel_dp_vect[i], skel_dp_vect[i-1])
        if angle > 150:
            skel_dp_vect[i] = abs(skel_dp_vect[i]) * np.sign(skel_dp_vect[i-1])
    return skel_dp_vect

import math

def angle_between_vectors(v1, v2):
    dot_product = np.dot(v1, v2)
    magnitude_v1 = np.linalg.norm(v1)
    magnitude_v2 = np.linalg.norm(v2)
    cos_angle = dot_product / (magnitude_v1 * magnitude_v2)
    cos_angle = np.clip(cos_angle, -1, 1)  # Ensure within valid range for arccos
    angle_radians = np.arccos(cos_angle)
    angle_degrees = np.degrees(angle_radians)
    return angle_degrees

def flip_vectors_if_necessary(skel_dp_vect):
    """Flip the direction of vectors if the angle with the previous vector is > 150 degrees."""
    for i in range(1, len(skel_dp_vect)):
        angle = angle_between_vectors(skel_dp_vect[i], skel_dp_vect[i-1])
        if angle > 150:
            skel_dp_vect[i] = abs(skel_dp_vect[i]) * np.sign(skel_dp_vect[i-1])
    return skel_dp_vect


def quaternion_to_rotation_angle(q1, q2):
    # Normalize the quaternions
    q1 = q1 / np.linalg.norm(q1)
    q2 = q2 / np.linalg.norm(q2)
    
    # Compute the dot product
    dot_product = np.dot(q1, q2)
    
    # Ensure the dot product is within the valid range for arccos
    dot_product = np.clip(dot_product, -1.0, 1.0)
    
    # Calculate the angle in radians
    angle_rad = 2 * np.arccos(dot_product)
    
    # Convert the angle to degrees
    angle_deg = np.degrees(angle_rad)
    
    return angle_deg

def flip_quaternion_axis(quaternion):
    """Flip the axis of a quaternion by negating the x, y, and z components."""
    for i in range(1, len(quaternion)):
        angle = quaternion_to_rotation_angle(quaternion[i], quaternion[i-1])
        if angle > 130:
            quaternion[i] = np.array([-quaternion[i,0], -quaternion[i,1], -quaternion[i,2], quaternion[i,3]])
    return quaternion



In [12]:
# get the dotprops
skel_dp = navis.make_dotprops(skel_navis, k=20)
skel_dp.vect

array([[-0.34465352, -0.06719094,  0.9363222 ],
       [-0.34465352, -0.06719094,  0.9363222 ],
       [-0.34465352, -0.06719094,  0.9363222 ],
       [-0.34465355, -0.06719095,  0.9363222 ],
       [-0.34465352, -0.06719094,  0.9363222 ],
       [-0.34465352, -0.06719094,  0.9363222 ],
       [-0.34465352, -0.06719095,  0.9363222 ],
       [-0.34465352, -0.06719095,  0.9363222 ],
       [-0.34465352, -0.06719095,  0.9363222 ],
       [-0.34465352, -0.06719095,  0.9363222 ],
       [-0.34465352, -0.06719095,  0.9363222 ],
       [-0.3507406 , -0.08214332,  0.93286306],
       [-0.35692477, -0.09326065,  0.92946607],
       [-0.36109895, -0.10355673,  0.9267597 ],
       [-0.36344638, -0.10860612,  0.9252629 ],
       [-0.36434355, -0.11175617,  0.9245347 ],
       [-0.36608246, -0.11704367,  0.9231925 ],
       [-0.3655756 , -0.12504202,  0.92234427],
       [-0.36360973, -0.13451634,  0.9217881 ],
       [-0.36006093, -0.14301877,  0.92190117],
       [-0.35798514, -0.14715335,  0.922

In [13]:
skel_dp.vect = flip_vectors_if_necessary(skel_dp.vect)

In [14]:
# get the quaternion for each node
quaternions = []
for i in range(len(skel_dp)):
    quaternions.append(surface_normal_to_quaternion(skel_dp.vect[i]))
quaternions

[[0.03414340098165745, -0.17513735254783375, 0.0, 0.9839517955224949],
 [0.03414340098165745, -0.17513735254783375, 0.0, 0.9839517955224949],
 [0.03414340098165745, -0.17513735254783375, 0.0, 0.9839517955224949],
 [0.03414340178478878, -0.1751373523912616, 0.0, 0.9839517955224949],
 [0.03414340098165745, -0.17513735254783375, 0.0, 0.9839517955224949],
 [0.03414340098165745, -0.17513735254783375, 0.0, 0.9839517955224949],
 [0.034143404629080505, -0.17513735183676082, 0.0, 0.9839517955224949],
 [0.034143404629080505, -0.17513735183676082, 0.0, 0.9839517955224949],
 [0.034143404629080505, -0.17513735183676082, 0.0, 0.9839517955224949],
 [0.034143404629080505, -0.17513735183676082, 0.0, 0.9839517955224949],
 [0.034143404629080505, -0.17513735183676082, 0.0, 0.9839517955224949],
 [0.04177887545915498, -0.17839001439407748, 0.0, 0.9830724939341194],
 [0.04747498385673277, -0.18169505093830499, 0.0, 0.982208243893488],
 [0.05275328581677233, -0.183948989970637, 0.0, 0.9815191592242692],
 [0.0

In [15]:
# flip the quaternion if necessary
fixed_quaternions = flip_quaternion_axis(np.array(quaternions))
fixed_quaternions

array([[ 0.0341434 , -0.17513735,  0.        ,  0.9839518 ],
       [ 0.0341434 , -0.17513735,  0.        ,  0.9839518 ],
       [ 0.0341434 , -0.17513735,  0.        ,  0.9839518 ],
       [ 0.0341434 , -0.17513735,  0.        ,  0.9839518 ],
       [ 0.0341434 , -0.17513735,  0.        ,  0.9839518 ],
       [ 0.0341434 , -0.17513735,  0.        ,  0.9839518 ],
       [ 0.0341434 , -0.17513735,  0.        ,  0.9839518 ],
       [ 0.0341434 , -0.17513735,  0.        ,  0.9839518 ],
       [ 0.0341434 , -0.17513735,  0.        ,  0.9839518 ],
       [ 0.0341434 , -0.17513735,  0.        ,  0.9839518 ],
       [ 0.0341434 , -0.17513735,  0.        ,  0.9839518 ],
       [ 0.04177888, -0.17839001,  0.        ,  0.98307249],
       [ 0.04747498, -0.18169505,  0.        ,  0.98220824],
       [ 0.05275329, -0.18394899,  0.        ,  0.98151916],
       [ 0.05534702, -0.18521675,  0.        ,  0.98113784],
       [ 0.05696309, -0.18570909,  0.        ,  0.98095226],
       [ 0.05967898, -0.

In [16]:
# move along the axon
# set the initial state
with viewer.txn() as s:
    s.voxel_coordinates = [skel_navis.nodes['x'][0], skel_navis.nodes['y'][0], skel_navis.nodes['z'][0]]
    s.crossSectionOrientation = fixed_quaternions[0]

In [17]:
# do move to according to the list of positions and orientations in df
for i in range(len(skel_navis.nodes)):
    do_move_to([skel_navis.nodes['x'][i], skel_navis.nodes['y'][i], skel_navis.nodes['z'][i]], final_orientation = fixed_quaternions[i], seconds = 0.2)

In [80]:
# current position
with viewer.txn() as s:
    current_pos = [s.voxel_coordinates[2],s.voxel_coordinates[1],s.voxel_coordinates[0]]

In [81]:
# create a bbox around the current position
dimensions = [100, 100, 100]

# Calculate the min and max coordinates of the bounding box
min_coords = [current_pos[i] - dimensions[i] / 2 for i in range(3)]
max_coords = [current_pos[i] + dimensions[i] / 2 for i in range(3)]

# Create the bounding box as a dictionary or any other structure you prefer
bbox = {
    'min': min_coords,
    'max': max_coords
}
bbox

{'min': [1778.0, 1769.0, 457.0], 'max': [1878.0, 1869.0, 557.0]}

In [82]:
skel_id = []
for skel in skel_all_navis:
    skel.nodes["in_bbox"] = (
        skel.nodes["x"].between(bbox['min'][0], bbox['max'][0]) &
        skel.nodes["y"].between(bbox['min'][1], bbox['max'][1]) &
        skel.nodes["z"].between(bbox['min'][2], bbox['max'][2])
    )
    skel_inbox = navis.subset_neuron(skel, skel.nodes["in_bbox"])
    if skel_inbox.n_nodes > 0:
        skel_id.append(skel.name)
        print(skel.name)
skel_id = np.array(skel_id.astype(int))-1

4690
0109
5658


In [88]:
skel_id = np.array(skel_id.astype(int))-2
skel_id

array([4689,  108, 5657])

In [89]:
# make nearby segments visible
with viewer.txn() as s:
  s.layers['segmentation'].segments = skel_id
  # s.voxel_coordinates = root_pos

In [93]:
current_pos = [skel_navis.nodes['z'][0], skel_navis.nodes['y'][0], skel_navis.nodes['x'][0]]

# create a bbox around the current position
dimensions = [100, 100, 100]

# Calculate the min and max coordinates of the bounding box
min_coords = [current_pos[i] - dimensions[i] / 2 for i in range(3)]
max_coords = [current_pos[i] + dimensions[i] / 2 for i in range(3)]

# Create the bounding box as a dictionary or any other structure you prefer
bbox = {
    'min': min_coords,
    'max': max_coords
}

skel_id = []
for skel in skel_all_navis:
    skel.nodes["in_bbox"] = (
        skel.nodes["x"].between(bbox['min'][0], bbox['max'][0]) &
        skel.nodes["y"].between(bbox['min'][1], bbox['max'][1]) &
        skel.nodes["z"].between(bbox['min'][2], bbox['max'][2])
    )
    skel_inbox = navis.subset_neuron(skel, skel.nodes["in_bbox"])
    if skel_inbox.n_nodes > 0:
        skel_id.append(skel.name)
        print(skel.name)
skel_id = np.array(skel_id).astype(int)-1

3852
4690
0510
5544
0109
5658


In [91]:
skel_id = np.array(skel_id).astype(int)-1

In [94]:
skel_id

array([3851, 4689,  509, 5543,  108, 5657])

In [95]:
with viewer.txn() as s:
    s.voxel_coordinates = [skel_navis.nodes['x'][0], skel_navis.nodes['y'][0], skel_navis.nodes['z'][0]]
    s.crossSectionOrientation = fixed_quaternions[0]
    s.layers['segmentation'].segments = skel_id

In [45]:
# filter the skeletons that have a nodes that are within a certain distance from the current position
# get the distance of each node from the current position
dist = []
for i in range(len(skel_all_navis.nodes)):
    dist.append(math.sqrt((skel_all_navis.nodes['x'][i] - current_pos[2])**2 + (skel_all_navis.nodes['y'][i] - current_pos[1])**2 + (skel_all_navis.nodes['z'][i] - current_pos[0])**2))
dist

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f3f1b271ad0>>
Traceback (most recent call last):
  File "/home/wanqing.yu/anaconda3/envs/EM/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 
Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f3f1b271ad0>>
Traceback (most recent call last):
  File "/home/wanqing.yu/anaconda3/envs/EM/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


KeyboardInterrupt: 

In [63]:
from ipywidgets import Button, VBox, Output
from IPython.display import display
import time

# Initialize widgets
btn_pause = Button(description='Pause')
btn_continue = Button(description='Continue', disabled=True)
btn_back = Button(description='Back', disabled=True)
btn_forth = Button(description='Forth', disabled=True)
output = Output()

# Global control variables
i = 20
paused = False

# def do_move_to(position, final_orientation, seconds):
#     # Placeholder for the actual movement function
#     with output:
#         print(f"Moving to {position} with orientation {final_orientation} in {seconds} seconds")

def update_buttons():
    global i
    btn_back.disabled = i <= 0
    btn_forth.disabled = i >= len(skel_navis.nodes) - 1
    btn_continue.disabled = not paused

def on_pause_clicked(b):
    global paused
    paused = True
    update_buttons()

def on_continue_clicked(b):
    global paused
    paused = False
    update_buttons()

def on_back_clicked(b):
    global i
    if i > 0:
        i -= 1
        do_move_to([skel_navis.nodes['x'][i], skel_navis.nodes['y'][i], skel_navis.nodes['z'][i]], final_orientation=quaternions[i], seconds=1)
    update_buttons()

def on_forth_clicked(b):
    global i
    if i < len(skel_navis.nodes) - 1:
        i += 1
        do_move_to([skel_navis.nodes['x'][i], skel_navis.nodes['y'][i], skel_navis.nodes['z'][i]], final_orientation=quaternions[i], seconds=1)
    update_buttons()

# Attach event handlers
btn_pause.on_click(on_pause_clicked)
btn_continue.on_click(on_continue_clicked)
btn_back.on_click(on_back_clicked)
btn_forth.on_click(on_forth_clicked)

# Display widgets
display(VBox([btn_pause, btn_continue, btn_back, btn_forth, output]))

# Initial execution
do_move_to([skel_navis.nodes['x'][i], skel_navis.nodes['y'][i], skel_navis.nodes['z'][i]], final_orientation=quaternions[i], seconds=0.2)

In [99]:
from ipywidgets import Button, VBox, Output
from IPython.display import display
import threading
import time

# Initialize widgets
btn_pause = Button(description='Pause')
btn_continue = Button(description='Continue', disabled=True)
btn_back = Button(description='Back', disabled=True)
btn_forth = Button(description='Forth', disabled=True)
output = Output()

# Global control variables
i = 0
paused = False
stop_thread = False

# def do_move_to(position, final_orientation, seconds):
#     # Placeholder for the actual movement function
#     with output:
#         print(f"Moving to {position} with orientation {final_orientation} in {seconds} seconds")

def update_buttons():
    global i
    btn_back.disabled = i <= 0
    btn_forth.disabled = i >= len(skel_navis.nodes) - 1
    btn_continue.disabled = not paused
    btn_pause.disabled = paused

def on_pause_clicked(b):
    global paused
    paused = True
    update_buttons()

def on_continue_clicked(b):
    global paused
    paused = False
    update_buttons()

def on_back_clicked(b):
    global i
    if i > 0:
        i -= 1
        do_move_to([skel_navis.nodes['x'][i], skel_navis.nodes['y'][i], skel_navis.nodes['z'][i]], final_orientation=quaternions[i], seconds=1)
    update_buttons()

def on_forth_clicked(b):
    global i
    if i < len(skel_navis.nodes) - 1:
        i += 1
        do_move_to([skel_navis.nodes['x'][i], skel_navis.nodes['y'][i], skel_navis.nodes['z'][i]], final_orientation=quaternions[i], seconds=1)
    update_buttons()

def loop_through_positions():
    global i, paused, stop_thread
    while not stop_thread:
        if not paused and i < len(skel_navis.nodes):
            do_move_to([skel_navis.nodes['x'][i], skel_navis.nodes['y'][i], skel_navis.nodes['z'][i]], final_orientation=quaternions[i], seconds=0.5)
            i += 1
            update_buttons()
        time.sleep(0.1)  # Adjust the sleep time as needed

# Attach event handlers
btn_pause.on_click(on_pause_clicked)
btn_continue.on_click(on_continue_clicked)
btn_back.on_click(on_back_clicked)
btn_forth.on_click(on_forth_clicked)

# Display widgets
display(VBox([btn_pause, btn_continue, btn_back, btn_forth, output]))

# Start the loop in a separate thread
thread = threading.Thread(target=loop_through_positions)
thread.start()

# Initial execution
# do_move_to([skel_navis.nodes['x'][i], skel_navis.nodes['y'][i], skel_navis.nodes['z'][i]], final_orientation=quaternions[i], seconds=0.2)

KeyError: 152

In [ ]:
# loading all skeletons to 